In [1]:
import pandas as pd
from textblob import TextBlob
import nltk
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

In [3]:
#df=pd.read_csv("/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv")
df = pd.read_csv( "/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv", index_col=None, header=0, engine='python' )

In [4]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,1,Am I weird I don t get affected by compliments...,non-suicide
2,2,Finally is almost over So I can never hear...,non-suicide
3,3,i need helpjust help me im crying so hard,suicide
4,4,I m so lostHello my name is Adam and I ve b...,suicide
5,5,Honetly idkI dont know what im even doing here...,suicide
6,6,Trigger warning Excuse for self inflicted bu...,suicide
7,7,It ends tonight I can t do it anymore I quit,suicide
8,8,Everyone wants to be edgy and it s making me...,non-suicide
9,9,My life is over at years oldHello all I am a...,suicide


In [6]:
df = pd.read_csv('/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv',
                 lineterminator='\n')

In [7]:
l=[]
for i in range(10000):
    l.append(TextBlob(df.iloc[i,1]).sentiment.polarity)
l1=[]
for i in l:
    if i>=0.2:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[0:10000,2].tolist()


In [8]:
from sklearn.metrics import accuracy_score
print(accuracy_score(l1,l2))

0.5704


In [9]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/aman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:

def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df
#Generating Embeddings using tokenizer
df=df.iloc[0:100,:]
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)
#Model Building
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-08 19:07:53.844735: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-08 19:07:53.844766: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1580, 120)         60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 1580, 120)        0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 704)               2323200   
                                                                 
 dense (Dense)               (None, 352)               248160    
                                                                 
 dense_1 (Dense)             (None, 3)                 1059      
                                                                 
Total params: 2,632,419
Trainable params: 2,632,419
Non-trainable params: 0
______________________________________________

2023-02-08 19:07:54.351051: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-08 19:07:55.595960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 5s 5s/step - loss: 0.8183 - accuracy: 0.4500


[0.8182739019393921, 0.44999998807907104]

In [ ]:
print(X_train,len(X_train))
print(X_test,len(X_test))
print(y_train,len(y_train))
print(y_test,len(y_test))

[[  0   0   0 ...  52   2 332]
 [  0   0   0 ...   5   8   6]
 [  0   0   0 ...   2   4 330]
 ...
 [  0   0   0 ...   6  24   5]
 [  0   0   0 ...  76  25 221]
 [  0   0   0 ...  21   5 282]] 80
[[  0   0   0 ... 107   1   6]
 [  0   0   0 ...  13   5 240]
 [  0   0   0 ...  67 280 347]
 ...
 [  0   0   0 ... 161   9   6]
 [  0   0   0 ...   1 116  58]
 [  0   0   0 ...  26   9  21]] 20
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.

In [ ]:
df.iloc[0:100,1:2]

,text
0,ex wife threatening suiciderecently i left my ...
1,am i weird i don t get affected by compliment ...
2,finally is almost over so i can never hear ha ...
3,i need helpjust help me im cry so hard
4,i m so losthello my name is adam and i ve been...
...,...
95,day of posting something i get mildly infuriat...
96,a lot of stuff to handle at once for me sorry ...
97,i m going to wait for a version of senorita in...
98,you know i just realized you cant have an opin...


In [ ]:
from transformers import pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)
l1=[]
l1=sentiment_pipeline(df.iloc[0,1])
print(l1[0]["label"])


[{'label': 'NEGATIVE', 'score': 0.9910893440246582}]
NEGATIVE


In [ ]:
df.head(10)

,Unnamed: 0,text,class
0,0,ex wife threatening suiciderecently i left my ...,1
1,1,am i weird i don t get affected by compliment ...,0
2,2,finally is almost over so i can never hear ha ...,0
3,3,i need helpjust help me im cry so hard,1
4,4,i m so losthello my name is adam and i ve been...,1
5,5,honetly idki dont know what im even doing here...,1
6,6,trigger warning excuse for self inflicted burn...,1
7,7,it end tonight i can t do it anymore i quit,1
8,8,everyone want to be edgy and it s making me se...,0
9,9,my life is over at year oldhello all i am a ye...,1


In [14]:
for i in l1:
    if i>=0:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[0:100,2].tolist()
print(accuracy_score(l1,l2))

TypeError: '>=' not supported between instances of 'dict' and 'int'